In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from sklearn.impute import SimpleImputer, KNNImputer
import polars as pl
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
%matplotlib inline


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv', low_memory = False)
train_data = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test_data = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sub_sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')
sample = sub_sample.copy()
# print(sample)

train_data.head()
df.info()
df.Instrument


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Instrument    81 non-null     object
 1   Field         81 non-null     object
 2   Description   81 non-null     object
 3   Type          81 non-null     object
 4   Values        42 non-null     object
 5   Value Labels  32 non-null     object
dtypes: object(6)
memory usage: 3.9+ KB


0                             Identifier
1                           Demographics
2                           Demographics
3                           Demographics
4     Children's Global Assessment Scale
                     ...                
76               Sleep Disturbance Scale
77               Sleep Disturbance Scale
78               Sleep Disturbance Scale
79                          Internet Use
80                          Internet Use
Name: Instrument, Length: 81, dtype: object

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 82 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      3960 non-null   object 
 1   Basic_Demos-Enroll_Season               3960 non-null   object 
 2   Basic_Demos-Age                         3960 non-null   int64  
 3   Basic_Demos-Sex                         3960 non-null   int64  
 4   CGAS-Season                             2555 non-null   object 
 5   CGAS-CGAS_Score                         2421 non-null   float64
 6   Physical-Season                         3310 non-null   object 
 7   Physical-BMI                            3022 non-null   float64
 8   Physical-Height                         3027 non-null   float64
 9   Physical-Weight                         3076 non-null   float64
 10  Physical-Waist_Circumference            898 non-null    floa

In [4]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 59 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      20 non-null     object 
 1   Basic_Demos-Enroll_Season               20 non-null     object 
 2   Basic_Demos-Age                         20 non-null     int64  
 3   Basic_Demos-Sex                         20 non-null     int64  
 4   CGAS-Season                             10 non-null     object 
 5   CGAS-CGAS_Score                         8 non-null      float64
 6   Physical-Season                         14 non-null     object 
 7   Physical-BMI                            13 non-null     float64
 8   Physical-Height                         13 non-null     float64
 9   Physical-Weight                         13 non-null     float64
 10  Physical-Waist_Circumference            5 non-null      float64


In [5]:
train_data.isna().sum()

id                                           0
Basic_Demos-Enroll_Season                    0
Basic_Demos-Age                              0
Basic_Demos-Sex                              0
CGAS-Season                               1405
                                          ... 
SDS-SDS_Total_Raw                         1351
SDS-SDS_Total_T                           1354
PreInt_EduHx-Season                        420
PreInt_EduHx-computerinternet_hoursday     659
sii                                       1224
Length: 82, dtype: int64

In [6]:
train_data.shape

(3960, 82)

In [7]:
###Explore data analysis
## Categorical columns in dataset
categorical_columns = ['Basic_Demos-Enroll_Season',
'CGAS-Season',
'Physical-Season',
'Fitness_Endurance-Season',
'FGC-Season',
'BIA-Season',
'PAQ_A-Season',
'PAQ_C-Season',
'PCIAT-Season',
'SDS-Season',
'PreInt_EduHx-Season'
]
# Plotting boxplots for 'sii' against each categorical column


In [8]:
## Check columns are numeric type
for label, content in train_data.items():
    if pd.api.types.is_numeric_dtype(content):
        print(label)

Basic_Demos-Age
Basic_Demos-Sex
CGAS-CGAS_Score
Physical-BMI
Physical-Height
Physical-Weight
Physical-Waist_Circumference
Physical-Diastolic_BP
Physical-HeartRate
Physical-Systolic_BP
Fitness_Endurance-Max_Stage
Fitness_Endurance-Time_Mins
Fitness_Endurance-Time_Sec
FGC-FGC_CU
FGC-FGC_CU_Zone
FGC-FGC_GSND
FGC-FGC_GSND_Zone
FGC-FGC_GSD
FGC-FGC_GSD_Zone
FGC-FGC_PU
FGC-FGC_PU_Zone
FGC-FGC_SRL
FGC-FGC_SRL_Zone
FGC-FGC_SRR
FGC-FGC_SRR_Zone
FGC-FGC_TL
FGC-FGC_TL_Zone
BIA-BIA_Activity_Level_num
BIA-BIA_BMC
BIA-BIA_BMI
BIA-BIA_BMR
BIA-BIA_DEE
BIA-BIA_ECW
BIA-BIA_FFM
BIA-BIA_FFMI
BIA-BIA_FMI
BIA-BIA_Fat
BIA-BIA_Frame_num
BIA-BIA_ICW
BIA-BIA_LDM
BIA-BIA_LST
BIA-BIA_SMM
BIA-BIA_TBW
PAQ_A-PAQ_A_Total
PAQ_C-PAQ_C_Total
PCIAT-PCIAT_01
PCIAT-PCIAT_02
PCIAT-PCIAT_03
PCIAT-PCIAT_04
PCIAT-PCIAT_05
PCIAT-PCIAT_06
PCIAT-PCIAT_07
PCIAT-PCIAT_08
PCIAT-PCIAT_09
PCIAT-PCIAT_10
PCIAT-PCIAT_11
PCIAT-PCIAT_12
PCIAT-PCIAT_13
PCIAT-PCIAT_14
PCIAT-PCIAT_15
PCIAT-PCIAT_16
PCIAT-PCIAT_17
PCIAT-PCIAT_18
PCIAT-PCIAT_19

In [9]:
## Check columns have null data
for label, content in train_data.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)


CGAS-CGAS_Score
Physical-BMI
Physical-Height
Physical-Weight
Physical-Waist_Circumference
Physical-Diastolic_BP
Physical-HeartRate
Physical-Systolic_BP
Fitness_Endurance-Max_Stage
Fitness_Endurance-Time_Mins
Fitness_Endurance-Time_Sec
FGC-FGC_CU
FGC-FGC_CU_Zone
FGC-FGC_GSND
FGC-FGC_GSND_Zone
FGC-FGC_GSD
FGC-FGC_GSD_Zone
FGC-FGC_PU
FGC-FGC_PU_Zone
FGC-FGC_SRL
FGC-FGC_SRL_Zone
FGC-FGC_SRR
FGC-FGC_SRR_Zone
FGC-FGC_TL
FGC-FGC_TL_Zone
BIA-BIA_Activity_Level_num
BIA-BIA_BMC
BIA-BIA_BMI
BIA-BIA_BMR
BIA-BIA_DEE
BIA-BIA_ECW
BIA-BIA_FFM
BIA-BIA_FFMI
BIA-BIA_FMI
BIA-BIA_Fat
BIA-BIA_Frame_num
BIA-BIA_ICW
BIA-BIA_LDM
BIA-BIA_LST
BIA-BIA_SMM
BIA-BIA_TBW
PAQ_A-PAQ_A_Total
PAQ_C-PAQ_C_Total
PCIAT-PCIAT_01
PCIAT-PCIAT_02
PCIAT-PCIAT_03
PCIAT-PCIAT_04
PCIAT-PCIAT_05
PCIAT-PCIAT_06
PCIAT-PCIAT_07
PCIAT-PCIAT_08
PCIAT-PCIAT_09
PCIAT-PCIAT_10
PCIAT-PCIAT_11
PCIAT-PCIAT_12
PCIAT-PCIAT_13
PCIAT-PCIAT_14
PCIAT-PCIAT_15
PCIAT-PCIAT_16
PCIAT-PCIAT_17
PCIAT-PCIAT_18
PCIAT-PCIAT_19
PCIAT-PCIAT_20
PCIAT-PCIAT_Tota

In [10]:
def fill_numeric(frame) :
    from warnings import simplefilter
    simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
## Fill numeric rows with the median
    for label, content in frame.items():
        if pd.api.types.is_numeric_dtype(content):
           if pd.isnull(content).sum():
            #Add a binary column which tell us if the data was missing or not
            #missing_copy[label + "missing-value"] = pd.isnull(content)
            #Fill missing values with median
            frame[label] = content.fillna(content.median())
    return frame

In [11]:
def fill_category(frame):
    #Change object type to category
    for label,content in frame.items():
        if not pd.api.types.is_numeric_dtype(content):
            frame[label] = content.astype("category").cat.as_ordered()
    #Fill with number
    for label, content in frame.items():
        if not pd.api.types.is_numeric_dtype(content):
            frame[label]  = pd.Categorical(content).codes + 1

    return frame

# Actigraphy (time series)

In [12]:
actigraphy = pl.read_parquet('/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=0417c91e/part-0.parquet')
# train_copy = pl.from_pandas(train_copy)
actigraphy

step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
u32,f32,f32,f32,f32,f32,f32,f32,f32,i64,i8,i8,f32
0,0.014375,-0.020112,-0.995358,0.00106,-88.445251,0.0,41.0,4195.0,44100000000000,2,2,5.0
1,0.014167,-0.023278,-0.996164,0.000289,-88.3722,0.0,41.0,4194.833496,44105000000000,2,2,5.0
2,0.014036,-0.022964,-0.99632,0.000301,-88.356422,0.0,41.5,4194.666504,44110000000000,2,2,5.0
3,0.013593,-0.022048,-0.996762,0.002278,-88.575943,0.0,37.5,4194.5,44115000000000,2,2,5.0
4,-0.061772,-0.065317,-0.973063,0.092321,-88.391273,0.0,55.666668,4199.0,44780000000000,2,2,5.0
…,…,…,…,…,…,…,…,…,…,…,…,…
287174,-0.407433,0.091612,-0.377763,0.039733,-43.319416,0.0,7.0,3695.0,32875000000000,1,3,53.0
287175,-0.703572,0.016187,0.15956,0.03598,14.12139,0.0,7.0,3695.0,32880000000000,1,3,53.0
287176,-0.209607,-0.4697,0.636573,0.097799,44.998573,0.0,7.0,3695.0,32885000000000,1,3,53.0


# Feature Engineering

In [13]:
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)

    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df

def feature_engineering(df):
    season_cols = [col for col in df.columns if 'Season' in col]
    df = df.drop(season_cols, axis=1) 
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    
    return df

In [14]:
# Đọc dữ liệu
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

# Xử lý parquet thành các đặc trưng mới
train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

100%|██████████| 2/2 [00:00<00:00,  9.30it/s]


In [15]:

# Xử lý dữ liệu chuỗi thời gian
df_train = train_ts.drop('id', axis=1)
df_test = test_ts.drop('id', axis=1)
time_series_cols = train_ts.drop('id', axis=1).columns.tolist()

# Thêm cột 'id' vào các DataFrame
df_train["id"] = train_ts["id"]
df_test["id"] = test_ts["id"]

# Merge dữ liệu chuỗi thời gian vào train và test
train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, df_test, how="left", on='id')

test['sii'] = np.nan

train = feature_engineering(train)
train = train.dropna(thresh=10, axis=0)
test = feature_engineering(test)



featuresCols = ['Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-CGAS_Score', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-PAQ_A_Total',
                'PAQ_C-PAQ_C_Total', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T',
                'PreInt_EduHx-computerinternet_hoursday', 'sii', 'BMI_Age','Internet_Hours_Age','BMI_Internet_Hours',
                'BFP_BMI', 'FFMI_BFP', 'FMI_BFP', 'LST_TBW', 'BFP_BMR', 'BFP_DEE', 'BMR_Weight', 'DEE_Weight',
                'SMM_Height', 'Muscle_to_Fat', 'Hydration_Status', 'ICW_TBW']

featuresCols += time_series_cols

# Loại bỏ các cột chứa >= 50% giá trị bị thiếu
cols_to_remove = [col for col in featuresCols if train[col].isnull().mean() > 0.5]
featuresCols = [col for col in featuresCols if col not in cols_to_remove]

train = train[featuresCols]
train = train.dropna(subset='sii')

test = test[featuresCols]

# điền giá trị thiếu cho train data
imputer = KNNImputer(n_neighbors=5)
numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns
imputed_data = imputer.fit_transform(train[numeric_cols])
train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols)
train_imputed['sii'] = train_imputed['sii'].round().astype(int)
for col in train.columns:
    if col not in numeric_cols:
        train_imputed[col] = train[col]
        
train = train_imputed

# imputation cho test data
test['sii'] = np.nan
numeric_cols_test = test.select_dtypes(include=['float64', 'int64']).columns
imputed_test_data = imputer.transform(test[numeric_cols_test])
test_imputed = pd.DataFrame(imputed_test_data, columns=numeric_cols_test)
for col in test.columns:
    if col not in numeric_cols_test:
        test_imputed[col] = test[col]
        
test = test_imputed.drop('sii', axis=1)


In [16]:
train.isna().sum()

Basic_Demos-Age                           0
Basic_Demos-Sex                           0
CGAS-CGAS_Score                           0
Physical-BMI                              0
Physical-Height                           0
Physical-Weight                           0
Physical-Diastolic_BP                     0
Physical-HeartRate                        0
Physical-Systolic_BP                      0
FGC-FGC_CU                                0
FGC-FGC_CU_Zone                           0
FGC-FGC_PU                                0
FGC-FGC_PU_Zone                           0
FGC-FGC_SRL                               0
FGC-FGC_SRL_Zone                          0
FGC-FGC_SRR                               0
FGC-FGC_SRR_Zone                          0
FGC-FGC_TL                                0
FGC-FGC_TL_Zone                           0
BIA-BIA_Activity_Level_num                0
BIA-BIA_BMC                               0
BIA-BIA_BMI                               0
BIA-BIA_BMR                     

In [17]:
test.isna().sum()

Basic_Demos-Age                           0
Basic_Demos-Sex                           0
CGAS-CGAS_Score                           0
Physical-BMI                              0
Physical-Height                           0
Physical-Weight                           0
Physical-Diastolic_BP                     0
Physical-HeartRate                        0
Physical-Systolic_BP                      0
FGC-FGC_CU                                0
FGC-FGC_CU_Zone                           0
FGC-FGC_PU                                0
FGC-FGC_PU_Zone                           0
FGC-FGC_SRL                               0
FGC-FGC_SRL_Zone                          0
FGC-FGC_SRR                               0
FGC-FGC_SRR_Zone                          0
FGC-FGC_TL                                0
FGC-FGC_TL_Zone                           0
BIA-BIA_Activity_Level_num                0
BIA-BIA_BMC                               0
BIA-BIA_BMI                               0
BIA-BIA_BMR                     

In [18]:
df_train

,stat_0,stat_1,stat_2,stat_3,stat_4,stat_5,stat_6,stat_7,stat_8,stat_9,...,stat_87,stat_88,stat_89,stat_90,stat_91,stat_92,stat_93,stat_94,stat_95,id
0,50458.0,50458.0,50458.0,50458.0,50458.0,50458.0,50458.0,50458.0,50458.0,50458.0,...,5.314874,89.422226,0.0,2626.199951,4187.0,8.639500e+13,7.0,2.0,57.0,0745c390
1,340584.0,340584.0,340584.0,340584.0,340584.0,340584.0,340584.0,340584.0,340584.0,340584.0,...,3.966906,89.080330,1.0,2628.199951,4146.0,8.639500e+13,7.0,2.0,243.0,eaab7a96
2,40003.0,40003.0,40003.0,40003.0,40003.0,40003.0,40003.0,40003.0,40003.0,40003.0,...,5.066334,86.987267,0.0,2618.199951,4183.0,8.636500e+13,7.0,3.0,134.0,8ec2cc63
3,223915.0,223915.0,223915.0,223915.0,223915.0,223915.0,223915.0,223915.0,223915.0,223915.0,...,6.134459,89.976074,0.0,2502.000000,6000.0,8.639500e+13,7.0,4.0,72.0,b2987a65
4,15420.0,15420.0,15420.0,15420.0,15420.0,15420.0,15420.0,15420.0,15420.0,15420.0,...,2.774382,89.300034,0.0,1046.800049,4199.0,8.601500e+13,7.0,4.0,76.0,7b8842c3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,394128.0,394128.0,394128.0,394128.0,394128.0,394128.0,394128.0,394128.0,394128.0,394128.0,...,3.669502,89.025551,1.0,2576.399902,4191.0,8.639500e+13,7.0,4.0,161.0,cd68643b
992,1195.0,1195.0,1195.0,1195.0,1195.0,1195.0,1195.0,1195.0,1195.0,1195.0,...,1.786410,81.665283,0.0,1526.599976,4194.0,8.514000e+13,7.0,2.0,130.0,f8ff0bc8
993,393240.0,393240.0,393240.0,393240.0,393240.0,393240.0,393240.0,393240.0,393240.0,393240.0,...,3.692727,89.333710,1.0,2592.199951,4178.0,8.639500e+13,7.0,1.0,79.0,db23fbe4
994,40085.0,40085.0,40085.0,40085.0,40085.0,40085.0,40085.0,40085.0,40085.0,40085.0,...,1.673958,88.629547,0.0,1875.199951,4183.0,8.639500e+13,7.0,1.0,155.0,687c85e7


In [19]:
train

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,FGC-FGC_CU,...,FMI_BFP,LST_TBW,BFP_BMR,BFP_DEE,BMR_Weight,DEE_Weight,SMM_Height,Muscle_to_Fat,Hydration_Status,ICW_TBW
0,5.0,0.0,51.0,16.877316,46.0,50.8,80.2,79.8,123.8,0.0,...,0.332267,1.190475,8591.822097,13746.944840,18.356260,29.370079,0.424811,6.383063,0.643522,0.747453
1,9.0,0.0,60.4,14.035590,48.0,46.0,75.0,70.0,122.0,3.0,...,0.305154,1.458119,3719.320478,5950.914352,20.362087,32.579348,0.321056,12.718037,0.588157,0.777492
2,10.0,1.0,71.0,16.648696,56.5,75.6,65.0,94.0,117.0,20.0,...,0.234666,1.216586,13857.058232,21001.402970,15.344135,23.317770,0.496180,10.835650,0.603642,0.637492
3,9.0,0.0,71.0,18.292347,56.0,81.6,60.0,97.0,117.0,18.0,...,0.224196,1.281264,21298.377749,36207.411592,13.865564,23.571569,0.472854,6.274343,0.563684,0.661008
4,13.0,1.0,50.0,22.279952,59.5,112.2,60.0,73.0,102.0,12.0,...,0.198595,1.262516,90468.027355,135701.701175,11.862478,17.793672,0.594629,2.621003,0.562625,0.521399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2730,8.0,0.0,71.8,17.139810,52.5,67.2,60.0,65.0,112.0,0.0,...,0.255085,1.343482,15184.201563,25813.157324,15.405804,26.189881,0.385990,5.416433,0.546400,0.700249
2731,7.0,1.0,64.2,13.927006,48.5,46.6,65.0,75.0,105.0,0.0,...,0.292827,1.425131,1367.006219,1777.103699,20.735773,26.956438,0.373066,43.676843,0.644749,0.667565
2732,13.0,0.0,60.0,16.362460,59.5,82.4,71.0,70.0,104.0,16.0,...,0.198595,1.254711,13985.687504,23775.715110,14.646602,24.899272,0.500487,12.939628,0.641165,0.631642
2733,11.0,0.0,68.0,21.441500,60.0,109.8,79.0,99.0,116.0,15.0,...,0.195299,1.310804,42623.022658,68197.040233,11.418397,18.269490,0.479653,4.334530,0.496020,0.623919


In [20]:
train.isna().sum()

Basic_Demos-Age                           0
Basic_Demos-Sex                           0
CGAS-CGAS_Score                           0
Physical-BMI                              0
Physical-Height                           0
Physical-Weight                           0
Physical-Diastolic_BP                     0
Physical-HeartRate                        0
Physical-Systolic_BP                      0
FGC-FGC_CU                                0
FGC-FGC_CU_Zone                           0
FGC-FGC_PU                                0
FGC-FGC_PU_Zone                           0
FGC-FGC_SRL                               0
FGC-FGC_SRL_Zone                          0
FGC-FGC_SRR                               0
FGC-FGC_SRR_Zone                          0
FGC-FGC_TL                                0
FGC-FGC_TL_Zone                           0
BIA-BIA_Activity_Level_num                0
BIA-BIA_BMC                               0
BIA-BIA_BMI                               0
BIA-BIA_BMR                     

In [21]:
train.columns.tolist()

['Basic_Demos-Age',
 'Basic_Demos-Sex',
 'CGAS-CGAS_Score',
 'Physical-BMI',
 'Physical-Height',
 'Physical-Weight',
 'Physical-Diastolic_BP',
 'Physical-HeartRate',
 'Physical-Systolic_BP',
 'FGC-FGC_CU',
 'FGC-FGC_CU_Zone',
 'FGC-FGC_PU',
 'FGC-FGC_PU_Zone',
 'FGC-FGC_SRL',
 'FGC-FGC_SRL_Zone',
 'FGC-FGC_SRR',
 'FGC-FGC_SRR_Zone',
 'FGC-FGC_TL',
 'FGC-FGC_TL_Zone',
 'BIA-BIA_Activity_Level_num',
 'BIA-BIA_BMC',
 'BIA-BIA_BMI',
 'BIA-BIA_BMR',
 'BIA-BIA_DEE',
 'BIA-BIA_ECW',
 'BIA-BIA_FFM',
 'BIA-BIA_FFMI',
 'BIA-BIA_FMI',
 'BIA-BIA_Fat',
 'BIA-BIA_Frame_num',
 'BIA-BIA_ICW',
 'BIA-BIA_LDM',
 'BIA-BIA_LST',
 'BIA-BIA_SMM',
 'BIA-BIA_TBW',
 'SDS-SDS_Total_Raw',
 'SDS-SDS_Total_T',
 'PreInt_EduHx-computerinternet_hoursday',
 'sii',
 'BMI_Age',
 'Internet_Hours_Age',
 'BMI_Internet_Hours',
 'BFP_BMI',
 'FFMI_BFP',
 'FMI_BFP',
 'LST_TBW',
 'BFP_BMR',
 'BFP_DEE',
 'BMR_Weight',
 'DEE_Weight',
 'SMM_Height',
 'Muscle_to_Fat',
 'Hydration_Status',
 'ICW_TBW']

In [22]:
test.columns.tolist()

['Basic_Demos-Age',
 'Basic_Demos-Sex',
 'CGAS-CGAS_Score',
 'Physical-BMI',
 'Physical-Height',
 'Physical-Weight',
 'Physical-Diastolic_BP',
 'Physical-HeartRate',
 'Physical-Systolic_BP',
 'FGC-FGC_CU',
 'FGC-FGC_CU_Zone',
 'FGC-FGC_PU',
 'FGC-FGC_PU_Zone',
 'FGC-FGC_SRL',
 'FGC-FGC_SRL_Zone',
 'FGC-FGC_SRR',
 'FGC-FGC_SRR_Zone',
 'FGC-FGC_TL',
 'FGC-FGC_TL_Zone',
 'BIA-BIA_Activity_Level_num',
 'BIA-BIA_BMC',
 'BIA-BIA_BMI',
 'BIA-BIA_BMR',
 'BIA-BIA_DEE',
 'BIA-BIA_ECW',
 'BIA-BIA_FFM',
 'BIA-BIA_FFMI',
 'BIA-BIA_FMI',
 'BIA-BIA_Fat',
 'BIA-BIA_Frame_num',
 'BIA-BIA_ICW',
 'BIA-BIA_LDM',
 'BIA-BIA_LST',
 'BIA-BIA_SMM',
 'BIA-BIA_TBW',
 'SDS-SDS_Total_Raw',
 'SDS-SDS_Total_T',
 'PreInt_EduHx-computerinternet_hoursday',
 'BMI_Age',
 'Internet_Hours_Age',
 'BMI_Internet_Hours',
 'BFP_BMI',
 'FFMI_BFP',
 'FMI_BFP',
 'LST_TBW',
 'BFP_BMR',
 'BFP_DEE',
 'BMR_Weight',
 'DEE_Weight',
 'SMM_Height',
 'Muscle_to_Fat',
 'Hydration_Status',
 'ICW_TBW']

In [23]:
train_copy = train.copy()

train_copy.dropna(subset = ["sii"], inplace = True)
train_copy.isna().sum()

Basic_Demos-Age                           0
Basic_Demos-Sex                           0
CGAS-CGAS_Score                           0
Physical-BMI                              0
Physical-Height                           0
Physical-Weight                           0
Physical-Diastolic_BP                     0
Physical-HeartRate                        0
Physical-Systolic_BP                      0
FGC-FGC_CU                                0
FGC-FGC_CU_Zone                           0
FGC-FGC_PU                                0
FGC-FGC_PU_Zone                           0
FGC-FGC_SRL                               0
FGC-FGC_SRL_Zone                          0
FGC-FGC_SRR                               0
FGC-FGC_SRR_Zone                          0
FGC-FGC_TL                                0
FGC-FGC_TL_Zone                           0
BIA-BIA_Activity_Level_num                0
BIA-BIA_BMC                               0
BIA-BIA_BMI                               0
BIA-BIA_BMR                     

In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Đảm bảo tái tạo kết quả
np.random.seed(42)

# Giả sử `train_copy` là dữ liệu huấn luyện và `test` là dữ liệu kiểm tra
# Kiểm tra dữ liệu đầu vào
if "sii" not in train_copy.columns:
    raise ValueError("Cột 'sii' không tồn tại trong dữ liệu huấn luyện!")

# # Mã hóa các cột dạng chuỗi trong `train_copy`
# for col in train_copy.columns:
#     if train_copy[col].dtype == 'object':
#         le = LabelEncoder()
#         train_copy[col] = le.fit_transform(train_copy[col].astype(str))

# season_columns = [col for col in train_copy.columns if 'season' in col.lower()]


# imputer = SimpleImputer(strategy='most_frequent')
# imputer.fit_transform(train_copy[season_columns])

# test[season_columns] = imputer.transform(test[season_columns])

# # Chuyển đổi `sii` thành số nguyên
# train_copy["sii"] = train_copy["sii"].astype(int)

# # Lấy danh sách các cột đặc trưng (không bao gồm `sii`)



In [25]:
# # Chuyển nhãn sang dạng số
# le = LabelEncoder()
# for col in season_columns:
#     train_copy[col] = le.fit_transform(train_copy[col])
#     test[col] = le.fit_transform(test[col])



In [26]:
train_copy.info

<bound method DataFrame.info of       Basic_Demos-Age  Basic_Demos-Sex  CGAS-CGAS_Score  Physical-BMI  \
0                 5.0              0.0             51.0     16.877316   
1                 9.0              0.0             60.4     14.035590   
2                10.0              1.0             71.0     16.648696   
3                 9.0              0.0             71.0     18.292347   
4                13.0              1.0             50.0     22.279952   
...               ...              ...              ...           ...   
2730              8.0              0.0             71.8     17.139810   
2731              7.0              1.0             64.2     13.927006   
2732             13.0              0.0             60.0     16.362460   
2733             11.0              0.0             68.0     21.441500   
2734             13.0              0.0             70.0     12.235895   

      Physical-Height  Physical-Weight  Physical-Diastolic_BP  \
0                46.0     

In [27]:
train_copy.isna().sum()

Basic_Demos-Age                           0
Basic_Demos-Sex                           0
CGAS-CGAS_Score                           0
Physical-BMI                              0
Physical-Height                           0
Physical-Weight                           0
Physical-Diastolic_BP                     0
Physical-HeartRate                        0
Physical-Systolic_BP                      0
FGC-FGC_CU                                0
FGC-FGC_CU_Zone                           0
FGC-FGC_PU                                0
FGC-FGC_PU_Zone                           0
FGC-FGC_SRL                               0
FGC-FGC_SRL_Zone                          0
FGC-FGC_SRR                               0
FGC-FGC_SRR_Zone                          0
FGC-FGC_TL                                0
FGC-FGC_TL_Zone                           0
BIA-BIA_Activity_Level_num                0
BIA-BIA_BMC                               0
BIA-BIA_BMI                               0
BIA-BIA_BMR                     

In [28]:
test.isna().sum()

Basic_Demos-Age                           0
Basic_Demos-Sex                           0
CGAS-CGAS_Score                           0
Physical-BMI                              0
Physical-Height                           0
Physical-Weight                           0
Physical-Diastolic_BP                     0
Physical-HeartRate                        0
Physical-Systolic_BP                      0
FGC-FGC_CU                                0
FGC-FGC_CU_Zone                           0
FGC-FGC_PU                                0
FGC-FGC_PU_Zone                           0
FGC-FGC_SRL                               0
FGC-FGC_SRL_Zone                          0
FGC-FGC_SRR                               0
FGC-FGC_SRR_Zone                          0
FGC-FGC_TL                                0
FGC-FGC_TL_Zone                           0
BIA-BIA_Activity_Level_num                0
BIA-BIA_BMC                               0
BIA-BIA_BMI                               0
BIA-BIA_BMR                     

In [29]:
# Chuẩn bị dữ liệu huấn luyện
com_columns = train.columns.intersection(test.columns).tolist()
X = train_copy[com_columns]
y = train_copy["sii"]


In [30]:
X.isna().sum()

Basic_Demos-Age                           0
Basic_Demos-Sex                           0
CGAS-CGAS_Score                           0
Physical-BMI                              0
Physical-Height                           0
Physical-Weight                           0
Physical-Diastolic_BP                     0
Physical-HeartRate                        0
Physical-Systolic_BP                      0
FGC-FGC_CU                                0
FGC-FGC_CU_Zone                           0
FGC-FGC_PU                                0
FGC-FGC_PU_Zone                           0
FGC-FGC_SRL                               0
FGC-FGC_SRL_Zone                          0
FGC-FGC_SRR                               0
FGC-FGC_SRR_Zone                          0
FGC-FGC_TL                                0
FGC-FGC_TL_Zone                           0
BIA-BIA_Activity_Level_num                0
BIA-BIA_BMC                               0
BIA-BIA_BMI                               0
BIA-BIA_BMR                     

In [31]:
# # Kiểm tra NaN hoặc Infinity trong X
# print("Kiểm tra NaN:", np.isnan(X).any())
# print("Kiểm tra Infinity:", np.isinf(X).any())

# # Hiển thị các giá trị lớn bất thường
# print("Giá trị lớn nhất:", np.max(X))

# Model

In [32]:
# Random forest

# Chia dữ liệu train/test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid_rf = {
    'n_estimators': [300],
    'max_depth': [10],
    'min_samples_split': [5]
} # thử [100, 200, 300], [None, 10, 20], [2, 5, 10]

# GridSearchCV
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, scoring='accuracy')
grid_rf.fit(X, y)

# Mô hình tốt nhất
rf_best = grid_rf.best_estimator_
print(f"Best Random Forest Parameters: {grid_rf.best_params_}")

Best Random Forest Parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 300}


In [33]:
# Catboost

cat = CatBoostClassifier(random_state=42, verbose=0)
param_grid_cat = {
    'iterations': [100],
    'depth': [4], 
    'learning_rate': [0.1]
} # thử [100, 200], [4, 6, 8], [0.01, 0.1, 0.2]

grid_cat = GridSearchCV(cat, param_grid_cat, cv=5, scoring='accuracy')
grid_cat.fit(X, y)

cat_best = grid_cat.best_estimator_
print(f"Best CatBoost Parameters: {grid_cat.best_params_}")

Best CatBoost Parameters: {'depth': 4, 'iterations': 100, 'learning_rate': 0.1}


In [34]:
# XGBoost

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
param_grid_xgb = {
    'n_estimators': [100],
    'max_depth': [4],
    'learning_rate': [0.01]
} #Thử [100, 200], [4, 6, 8], [0.01, 0.1, 0.2]

grid_xgb = GridSearchCV(xgb, param_grid_xgb, cv=5, scoring='accuracy')
grid_xgb.fit(X, y)

xgb_best = grid_xgb.best_estimator_
print(f"Best XGBoost Parameters: {grid_xgb.best_params_}")

Best XGBoost Parameters: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100}


In [35]:
# LGBM

# lgbm = LGBMClassifier(random_state=42)
# param_grid_lgbm = {
#     'n_estimators': [100],
#     'max_depth': [4],
#     'learning_rate': [0.1]
# } # thử [100, 200], [4, 6, 8], [0.01, 0.1, 0.2]

# grid_lgbm = GridSearchCV(lgbm, param_grid_lgbm, cv=3, scoring='accuracy')
# grid_lgbm.fit(X_train, y_train)

# lgbm_best = grid_lgbm.best_estimator_
# print(f"Best LightGBM Parameters: {grid_lgbm.best_params_}")

In [36]:
# Huấn luyện mô hình với tham số tốt nhất
rf_best.fit(X, y)
cat_best.fit(X, y)
xgb_best.fit(X, y)
# lgbm_best.fit(X_train, y_train)

# Lấy xác suất dự đoán
X_test_1 = test[com_columns]
rf_proba = rf_best.predict_proba(X_test_1)
cat_proba = cat_best.predict_proba(X_test_1)
xgb_proba = xgb_best.predict_proba(X_test_1)
# lgbm_proba = lgbm_best.predict_proba(X_test_1)


# Averaging Ensemble với trọng số

In [37]:
# Đặt trọng số cho từng mô hình (có thể tối ưu hóa trọng số)
weights = [0.3, 0.3, 0.4]  # Ví dụ trọng số cho RF, CatBoost, XGBoost, LightGBM

# Tính trung bình trọng số của xác suất
ensemble_proba = (
    weights[0] * rf_proba +
    weights[1] * cat_proba +
    weights[2] * xgb_proba
)

# Lấy nhãn dự đoán từ xác suất trung bình
y_test_1 = np.argmax(ensemble_proba, axis=1)


In [38]:
print("Predicted values for test data:", y_test_1)

Predicted values for test data: [0 0 0 0 1 0 0 0 2 1 0 0 1 0 1 0 0 0 0 1]


# Result

In [39]:
submission = sub_sample.copy()
submission['sii'] = y_test_1
submission.to_csv('submission.csv', index = False)
print(submission)

          id  sii
0   00008ff9    0
1   000fd460    0
2   00105258    0
3   00115b9f    0
4   0016bb22    1
5   001f3379    0
6   0038ba98    0
7   0068a485    0
8   0069fbed    2
9   0083e397    1
10  0087dd65    0
11  00abe655    0
12  00ae59c9    1
13  00af6387    0
14  00bd4359    1
15  00c0cd71    0
16  00d56d4b    0
17  00d9913d    0
18  00e6167c    0
19  00ebc35d    1
